In [1]:
import sys
sys.path.append("C:\\Users\\lauth\\OneDrive\\Desktop\\open_ai_assistant_v2")
from src.components.chains.chains import Assistant
import pandas as pd

assistant = Assistant(conversation_id="8af20de4-cd11-41da-9a9d-04762dd91322")

#### Generando un resumen de la conversación

In [2]:
assistant.add_user_message(message="Dame el promedio de la presion estatica del computador de flujo con tag FQI-EMED-3135.01-067 para los meses de agosto del 2023")
# assistant.add_ai_message(message="Aqui tienes la lista de los sistemas de Medicion: Medidor 1a, Medidor 2a, Medidor 3a")
# assistant.add_user_message(message="Hola")
# assistant.add_ai_message(message="Claro, ¿me puedes proporcionar un identificador del sistema de medición?")
# assistant.add_user_message(message="me dices cuales son los ids que existen?")
# assistant.add_user_message(message="el Tag es F980-40")
# assistant.add_ai_message(message="Por supuesto, ¿que tipo de información requieres?")
# assistant.add_user_message(message="Me gustaria saber el promedio de la presion en las fechas de enero y febrero")
# assistant.add_user_message(message="Necesito una lista de los sistemas de medicion de gas")
summary = assistant.get_memory_summary()
print(summary)


        The human is asking for the average static pressure of the flow computer with Tag FQI-EMED-3135.01-067 for the months of August 2023.


#### Generamos el requerimiento

In [3]:
user_request = assistant.get_request(summary)
print(user_request["request"])

The human is asking for the average static pressure of the flow computer with Tag FQI-EMED-3135.01-067 for the months of August 2023.


#### Obtenemos las keywords

In [4]:
kw = assistant.get_keywords_from_requirement(user_request["request"])
kw

['variables', 'pressure', 'flow computer', 'average', 'month', '2023']

#### Nombre de tablas

In [5]:
r = assistant.get_table_names(kw)
print(r["tables"])
print(r["ddl"])


['fcs_computador_medidor', 'var_variable_datos', 'fcs_computadores']
['CREATE TABLE IF NOT EXISTS dbo_v2.var_variable_datos (\n    Fecha DATETIME,\n    idVariable_fk INT,\n    idSistemaMedicion_fk INT,\n    Valor FLOAT,\n    Valor_String VARCHAR(100),\n    PRIMARY KEY (Fecha, idVariable_fk, idSistemaMedicion_fk),\n    FOREIGN KEY (idVariable_fk) REFERENCES dbo_v2.var_tipo_variable (Id),\n    FOREIGN KEY (idSistemaMedicion_fk) REFERENCES dbo_v2.med_sistema_medicion (Id)\n)', 'CREATE TABLE IF NOT EXISTS dbo_v2.fcs_computador_medidor (\n    Id INT PRIMARY KEY,\n    Codigo_Medidor VARCHAR(100),\n    Estado VARCHAR(100),\n    Id_Sisema_Medicion INT,\n    Id_Sistema_Medicion_Redundante INT,\n    IdComputador_fk INT,\n    FOREIGN KEY (IdComputador_fk) REFERENCES dbo_v2.fcs_computadores (Id),\n    FOREIGN KEY (Id_Sisema_Medicion) REFERENCES dbo_v2.med_sistema_medicion (Id),\n    FOREIGN KEY (Id_Sistema_Medicion_Redundante) REFERENCES dbo_v2.med_sistema_medicion (Id)\n)', 'CREATE TABLE IF NOT E

#### Clasificamos el requerimiento simple-complejo

In [6]:
request_type = assistant.simple_complex_filter(user_request["request"])
print(request_type)

{'analysis': 'This input is out of my knowledge, this type is complex', 'response': 'complex'}


#### Clasificamos el requerimiento complejo (completo-incompleto)

In [9]:
request_type = assistant.complete_incomplete_filter(user_request["request"], r["ddl"])
print(request_type)


        Your are an assistant and your work is to classify the input to complete or incomplete.
        
        This request will query the next SQL DDL's
        DDL's:
        CREATE TABLE IF NOT EXISTS dbo_v2.var_variable_datos (
    Fecha DATETIME,
    idVariable_fk INT,
    idSistemaMedicion_fk INT,
    Valor FLOAT,
    Valor_String VARCHAR(100),
    PRIMARY KEY (Fecha, idVariable_fk, idSistemaMedicion_fk),
    FOREIGN KEY (idVariable_fk) REFERENCES dbo_v2.var_tipo_variable (Id),
    FOREIGN KEY (idSistemaMedicion_fk) REFERENCES dbo_v2.med_sistema_medicion (Id)
)

CREATE TABLE IF NOT EXISTS dbo_v2.fcs_computador_medidor (
    Id INT PRIMARY KEY,
    Codigo_Medidor VARCHAR(100),
    Estado VARCHAR(100),
    Id_Sisema_Medicion INT,
    Id_Sistema_Medicion_Redundante INT,
    IdComputador_fk INT,
    FOREIGN KEY (IdComputador_fk) REFERENCES dbo_v2.fcs_computadores (Id),
    FOREIGN KEY (Id_Sisema_Medicion) REFERENCES dbo_v2.med_sistema_medicion (Id),
    FOREIGN KEY (Id_Sistema_Med

In [10]:
print(request_type["text"])


        analysis: The requirement is incomplete for the db because it does not specify which table to query for the average static pressure of the flow computer with Tag FQI-EMED-3135.01-067 for the months of August 2023.
        type: incomplete
